In [21]:
import aspose.slides as slides
import os
import shutil
from pydub import AudioSegment
import whisper
import glob
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from aspose.slides import AudioFrame

AudioSegment.converter = "C:\\PATH_Programs\\ffmpeg.exe"
AudioSegment.ffmpeg = "C:\\PATH_Programs\\ffmpeg.exe"
AudioSegment.ffprobe ="C:\\PATH_Programs\\ffprobe.exe"

In [22]:
presentations_folder = "presentations"
if not os.path.exists(presentations_folder):
    os.makedirs(presentations_folder)

transcriptions_folder = "transcriptions"
if not os.path.exists(transcriptions_folder):
    os.makedirs(transcriptions_folder)

In [23]:
temp_folder = "audio_files"

# Delete the output folder if it exists so it empties
if os.path.exists(temp_folder):
    shutil.rmtree(temp_folder)

# Create the output folder if it doesn't exist (should run everytime)
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

In [25]:
ppt_files = glob.glob(presentations_folder + "/*.pptx")

In [26]:
#"tiny" = 32x speed - worst accuracy
#"base" = 16x speed - recommended
#"small" = 6x speed - best accuracy
model_type = "small"
model = whisper.load_model(model_type)

In [28]:
def create_pdf(results, output_file):
    # Create a PDF document
    doc = SimpleDocTemplate(output_file, pagesize=letter)

    # Define styles for paragraphs
    styles = getSampleStyleSheet()
    style = styles["Normal"]

    # Add each result as a paragraph to the document
    paragraphs = []
    for result in results:
        # Add the result as a paragraph to the document
        p = Paragraph(result.strip(), style)
        paragraphs.append(p)
        
        # Add a single line break to separate items
        paragraphs.append(Spacer(1, 12))  # Adjust the second argument to control the spacing

    # Build the PDF document
    doc.build(paragraphs)

In [33]:
for ppt in ppt_files:
    # Delete the output folder if it exists so it empties
    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)
    # Create the output folder if it doesn't exist (should run everytime)
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder)
    
    #input powerpoint presentation name
    parts = ppt.split('.')
    part_without_extension = parts[0]
    parts = part_without_extension.split('\\')
    final_name = parts[1]
    output_name = "transcriptions/" + final_name + ".pdf"

    #main code
    with slides.Presentation(ppt) as presentation:
        for index, audio in enumerate(presentation.audios):
            audio_data = audio.binary_data
            content_type = audio.content_type
            file_extension = content_type.split("/")[-1]  # Get the file extension from content type
            
            # Construct the file name
            file_name = f"audio_{index + 1}.{file_extension}"
            
            # Write the binary audio data to a file
            with open(os.path.join(temp_folder, file_name), "wb") as file:
                file.write(audio_data)
            
        print(f"Downloaded {final_name} audios to {temp_folder}")
    
    print(f"Transcribing {final_name} audios")
    results = []
    audio_files = [f for f in os.listdir(temp_folder) if os.path.isfile(os.path.join(temp_folder, f))]
    for file in audio_files:
        result = model.transcribe(os.path.join(temp_folder,file))
        res_text = result.get("text")
        results.append(res_text)

    #create the pdf
    create_pdf(results, output_name)
    print(f"PDF created for {final_name}: {output_name}\n")
    print("------------------------------------------------------")

Downloaded Lecture 10_Corporate Culture audios to audio_files
Transcribing Lecture 10_Corporate Culture audios
PDF created for presentation presentations\Lecture 10_Corporate Culture.pptx: transcriptions/Lecture 10_Corporate Culture.pdf
Downloaded Lecture 11_Changing Corporate Culture audios to audio_files
Transcribing Lecture 11_Changing Corporate Culture audios
PDF created for presentation presentations\Lecture 11_Changing Corporate Culture.pptx: transcriptions/Lecture 11_Changing Corporate Culture.pdf
Downloaded Lecture 1_Intro Business Anth audios to audio_files
Transcribing Lecture 1_Intro Business Anth audios
PDF created for presentation presentations\Lecture 1_Intro Business Anth.pptx: transcriptions/Lecture 1_Intro Business Anth.pdf
Downloaded Lecture 2_Culture Concept audios to audio_files
Transcribing Lecture 2_Culture Concept audios
PDF created for presentation presentations\Lecture 2_Culture Concept.pptx: transcriptions/Lecture 2_Culture Concept.pdf
Downloaded Lecture 3_Met